普通Linux系统下通过查看/proc/ioports文件可以得到相关控制器或设置使用的I/O地址范围
32kB-1KB=0x7c00(111 1100 0000 0000)

上电后，内存从主板rom中加载bios程序到固定位置，并自动执行；
bios程序加载中断向量表、硬盘参数等，将CPU初始化到可用状态。
将CPU cs寄存器地址为0xF000， 基段地址为0xFFFF0000，IP值为0xFFF0，所以此时的代码地址是0xFFFF0000+0xFFF0=0xFFFFFFF0 。
BIOS提前在内存0xFFFFFFF0出存放一条JMP指令，跳转到内存中bios程序段的一个固定地址；
bios此处代码会经过一系列硬件检测和初始化操作后，会将硬盘引导扇区的512B数据导入到0x7c00处。
引导扇区中的代码把硬盘的分区表信息载入到内存，查找标记为活动的分区，并在该分区中加载执行grub引导程序的代码；
grub启动后，加载文件系统，找到配置文件grub.cfg; grub.cfg指出了内核的在文件中的位置


bootsec把setup和system（头部代码为head）加载到内存，把自己和setup移动到0x90000和0x90200之后，执行权交给setup。setup利用bios中的中断获取磁盘、显卡参数，然后把system移动到0x00000处，然后开始加载GDT、IDT。然后重新设置中断控制器，设置CR0进入保护模式，跳转到head代码出执行。head初始化中断描述符，初始化内存，最后跳转到system的init/main继续执行。

main获取setup取得的机器参数，包括根文件设备号，内存大小和地址。然后开始初始化硬件，包括陷阱们、块设备、字符设备、tty。启动task 0，开启中断并切换到task 0 。此时初始化工作结束。然后task 0创建task 1，进入到task1， task0 执行pause等待空闲。
task1 加载文件系统、设置虚拟盘， 初始化tty0，显示系统信息；fork出task2，task2执行rc命令，根据/etc/rc的配置执行相关命令；task2结束后，再次fork， 生成大循环；
每个循环执行getty，getty初始化终端类型，提示用户输入用户名，提示信息可由/etc/issue编写；
用户输入用户名后，调用login进程，login进程读取/etc/passwd获取用户的登录项；然后调用getpass（）提示用户输入密码，密码失败多次，login退出，回到大循环，继续等待输入用户名；
密码成功，设置环境变量，开启shell进程；shell读取/etc/profile文件，执行预设命令；每次执行shell需要执行的命令放到/etc/profile；每次登录需要执行的命令，放到$HOME/.xxxx

linux设备分为字符（char）、块（block）、网络（socket）
主设备号，1对应/dev/null、/dev/zero；  4对应虚拟控制台和串口终端驱动程序； 7对应vcsl和vcsal

```shell
`# scrull_load`
#!/bin/sh
module="scull"
device="scull"
mode="664"

/sbin/insmod ./$module.ko $* || exit 1

rm -f /dev/${device}[0-3]

major=$(awk "\$2==\"$module\" {print \$1}" /porc/device)

mknod /dev/${device}0 c $major 0
mknod /dev/${device}1 c $major 1
mknod /dev/${device}2 c $major 2
mknod /dev/${device}3 c $major 3

group="staff"
grep -q '^staff:' /etc/group || group="wheel"

chgrp $group /dev/${device}[0-3]
chmod $mode /dev/${device}[0-3]
```